In [1]:
import pydicom
import glob
import os
import zipfile
import shutil

This script will anonimize dicom files, open zip files and copy all other files in a directory maintaining its structure.

In [2]:
def main(args):
    input_path = os.path.join(args.input_dir, args.glob)
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    
    files = glob.glob(input_path)

    for i, input_file in enumerate(files):
        try:
            # if pydicom can read the file, try to anonimize it and save it without the attributes specified in the argument class
            # feel free to delete this try expect cause if your data is already anonimized, or do not need anonimization
            dicom_file = pydicom.dcmread(input_file)
            for attribute in args.required_attributes:
                if attribute in dicom_file:
                    dicom_file.data_element(attribute).value=''
            output_path = os.path.join(args.output_dir, os.path.basename(input_file))
            dicom_file.save_as(output_path)                 
        except:
            args2=argument_class()
            args2.input_dir=input_file
            args2.output_dir=os.path.join(args.output_dir, os.path.basename(input_file))
            if os.path.isdir(input_file):
                #if the current item is a directory, call the same function iteratively
                main(args2)
            elif input_file[len(input_file)-4:]=='.zip':
                #if the current item is a zip file, open it, move it to a temporary loaction, and call the main function on it.
                if not os.path.exists(args.temporary_dir):        
                    os.makedirs(args.temporary_dir)
                with zipfile.ZipFile(input_file, 'r') as zip:
                    zip.extractall(args.temporary_dir)
                args2.input_dir=args.temporary_dir
                args2.output_dir=args2.output_dir[:len(args2.output_dir)-4]
                main(args2)
                shutil.rmtree(args.temporary_dir)
            else:
                #if the current item is not a dicom, a zip or a directory, just copy it
                output_path = os.path.join(args.output_dir, os.path.basename(input_file))
                shutil.copyfile(input_file,output_path)


In [3]:
class argument_class:
    def __init__(self):
        self.input_dir=''
        self.output_dir=''
        self.glob='*'
        self.required_attributes=['PatientName','PatientBirthDate'] #these attrinutes will be nulled in the dicom files, thus anonimizing the dicom
        self.temporary_dir='C:\\Users\\Balint\\Desktop\\Temporary'  #tempopary directory to extract zip files to

In [4]:
args=argument_class()

In [5]:
args.input_dir='F:\\Natasa\\NRS'
args.output_dir='E:\\NRS_anon'

In [6]:
main(args)